Installing required libraries.

In [ ]:
!pip install pycaret

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install 'rich[jupyter]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.1 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Wed Mar 27 16:37:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              41W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 72.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [ ]:
import re

# Upload the data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving clarity_train.csv to clarity_train.csv


In [ ]:
uploaded2 = files.upload()

Saving clarity_test_public.csv to clarity_test_public.csv
Saving empathy_test_public.csv to empathy_test_public.csv
Saving fairness_test_public.csv to fairness_test_public.csv
Saving interview_test_public.csv to interview_test_public.csv


In [ ]:
import pandas as pd
clarity_train = pd.read_csv('clarity_train.csv')
clarity_test = pd.read_csv('clarity_test_public.csv')
empathy_df = pd.read_csv('empathy_test_public.csv')
fairness_df = pd.read_csv('fairness_test_public.csv')
df = pd.read_csv('interview_test_public.csv')

In [ ]:
from rich import print

# Clarity

In [ ]:
# Lowercase the training text
clarity_train['processed_personality_item'] = clarity_train['personality_item'].apply(lambda x: x.lower())

# Remove punctuation and numbers from the training text
clarity_train['processed_personality_item'] = clarity_train['processed_personality_item'].apply(lambda x: re.sub(r'[\d\W]+', ' ', x))

In [ ]:
# Lowercase the test text
clarity_test['processed_personality_item'] = clarity_test['personality_item'].apply(lambda x: x.lower())

# Remove punctuation and numbers from the test text
clarity_test['processed_personality_item'] = clarity_test['processed_personality_item'].apply(lambda x: re.sub(r'[\d\W]+', ' ', x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
# Fit and transform the training data on the processed text
tfidf_train = tfidf_vectorizer.fit_transform(clarity_train['processed_personality_item'])

# Only transform the test data on the processed text
tfidf_test = tfidf_vectorizer.transform(clarity_test['processed_personality_item'])

# Convert the TF-IDF transformation to a DataFrame for training data
tfidf_train_df = pd.DataFrame(tfidf_train.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_train_df['clarity'] = clarity_train['clarity'].values

# For the test data, just convert TF-IDF transformation to DataFrame and append the '_id' column
tfidf_test_df = pd.DataFrame(tfidf_test.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_test_df['_id'] = clarity_test['_id'].values

In [ ]:
from pycaret.regression import *

In [ ]:
# Set up the regression experiment
exp_reg1 = setup(data=tfidf_train_df,
                 target='clarity',
                 session_id=42,
                 ignore_features=['_id'])

,Description,Value
0,Session id,42
1,Target,clarity
2,Target type,Regression
3,Original data shape,"(30, 146)"
4,Transformed data shape,"(30, 146)"
5,Transformed train set shape,"(21, 146)"
6,Transformed test set shape,"(9, 146)"
7,Ignore features,1
8,Numeric features,145
9,Preprocess,True


In [ ]:
# Compare models and finalize the best
best = compare_models()
final_model = finalize_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,0.8616,1.0749,0.9204,-104.7931,0.1597,0.2037,0.1580
et,Extra Trees Regressor,0.7961,0.9827,0.9025,-140.4059,0.1503,0.1749,0.2170
ada,AdaBoost Regressor,0.7817,1.0921,0.8890,-226.5331,0.1487,0.1830,0.0870
knn,K Neighbors Regressor,0.7841,0.9265,0.8820,-247.8037,0.1547,0.1747,0.0350
lightgbm,Light Gradient Boosting Machine,1.2778,2.0297,1.3634,-289.9878,0.2384,0.2925,0.0850
lasso,Lasso Regression,1.2778,2.0297,1.3634,-289.9881,0.2384,0.2925,0.0540
dummy,Dummy Regressor,1.2778,2.0297,1.3634,-289.9881,0.2384,0.2925,0.0210
llar,Lasso Least Angle Regression,1.2778,2.0297,1.3634,-289.9881,0.2384,0.2925,0.0240
en,Elastic Net,1.2778,2.0297,1.3634,-289.9881,0.2384,0.2925,0.0250
ridge,Ridge Regression,1.1910,1.7714,1.2805,-370.8743,0.2246,0.2716,0.0250


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
# Make predictions on the test data
predictions = predict_model(final_model, data=tfidf_test_df)

In [ ]:
# To ensure the '_id' column is included in your final predictions DataFrame
final_predictions = predictions[['prediction_label']]
final_predictions['benchmark'] = 'clarity'
final_predictions['_id'] = tfidf_test_df['_id'].values

In [ ]:
final_predictions['output'] = final_predictions.pop('prediction_label')

In [ ]:
final_predictions.head()

,benchmark,_id,output
0,clarity,130,5.631743
1,clarity,131,3.352298
2,clarity,132,3.322012
3,clarity,133,6.284932
4,clarity,134,3.328331


In [ ]:
final_predictions.to_csv('clarity_output.csv', index=False)
files.download('clarity_output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Empathy

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key = "API-KEY-HERE"
)

In [ ]:
system_prompt = """You are an HR manager in charge of determining whether emails exhibit empathy or not. When evaluating the empathy of an email, consider the following heuristics:\n\nCharacteristics of less empathetic emails:\n\n- Lack of personalized, positive feedback and explicit guidance.\n- Direct and implicit criticism without constructive direction\n- Pressure, both implicit and explicit, without offering support.\n- Prioritizing organizational needs over individuality and stakeholder feedback.\n- Emphasizing efficiency and compliance over personal development and dialogue.\n- Failure to recognize or invite collaborative communication and dialogue.\n- Making unclarified assumptions and neglecting the emotional impact of messages.\n- Using directive language, imposing solutions, and mandating training without consensus.\n- Vagueness in feedback, goals, and the highlighting of public consequences of errors.\n- Formal or informal communication that mismatches the seriousness of feedback.\n- Neglecting to acknowledge efforts, emphasizing mistakes, and undermining achievements.\n- Imposing urgent deadlines, additional oversight, and repetitive tasks without clear support.\n- Conditional support for career growth and veiled critiques that imply inadequacy.\n- Communication errors such as mixed messages or compliment sandwiches.\n\nInstructions for Evaluation:\n\nRead each email message attentively. Balance your evaluation based on the heuristics for both empathetic and less empathetic characteristics. If an email does not definitively show several empathetic traits or displays numerous characteristics of less empathetic communication, classify it as lacking empathy. Your determination should be made with consideration to the nuanced balance between these traits. Respond with 'Choice: 0' for emails that do not exhibit empathy and 'Choice: 1' for emails that do. Provide a brief justification for your choice, focusing on the key aspects of the message that influenced your decision."""

In [ ]:
empathy_df = pd.read_csv('empathy_test_public.csv')

In [ ]:
# Generate a response from the custom model using the system and user prompts
# while also including the task and id
def generate_response(system_prompt, user_prompt, task, id):
  completion = client.chat.completions.create(
    model = "ft:gpt-3.5-turbo-1106:personal:empathy:94zGKA9B",
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}

    ])
  return task, id, completion.choices[0].message.content

In [ ]:
def parse_empathy_response(task, id, response):
# Use a regular expression to find the number right after the text 'Choice:'
  match = re.search(r'Choice:\s*(\d+)', response)
  if match:
    parsed_response = int(match.group(1))
  else:
    parsed_response = None
  return task, id, parsed_response

In [ ]:
# Create an empty DataFrame for the empathy responses
empathy_output_df = pd.DataFrame(columns=['benchmark', '_id', 'output'])

In [ ]:
empathy_output_df.head()

,benchmark,_id,output


In [ ]:
for index, row in empathy_df.iterrows():
  system_prompt = system_prompt
  user_prompt = row['text']
  task = 'empathy'
  _id = row['_id']

  # Generate the response
  task, _id, response = generate_response(system_prompt, user_prompt, task, _id)

  # Print the responses in case there's an API error
  # or if the parser doesn't parse correctly
  print("_id: ", _id, "response: ", response, "\n\n")

  # Parse the empathy response
  task, _id, parsed_empathy = parse_empathy_response(task, _id, response)

  # Convert the output tuple to a series
  new_row = pd.Series([task, _id, parsed_empathy], index=['benchmark', '_id', 'output'])

  # Append the series to the empathy output df
  empathy_output_df = empathy_output_df.append(new_row, ignore_index=True)

_id:  56 response:  Choice: 1 




<ipython-input-15-80a297121b42>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  empathy_output_df = empathy_output_df.append(new_row, ignore_index=True)


_id:  152 response:  Choice: 1 




<ipython-input-15-80a297121b42>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  empathy_output_df = empathy_output_df.append(new_row, ignore_index=True)


_id:  101 response:  Choice: 1 




...

<ipython-input-15-80a297121b42>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  empathy_output_df = empathy_output_df.append(new_row, ignore_index=True)


_id:  44 response:  Choice: 1 




<ipython-input-15-80a297121b42>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  empathy_output_df = empathy_output_df.append(new_row, ignore_index=True)


_id:  71 response:  Choice: 1 




<ipython-input-15-80a297121b42>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  empathy_output_df = empathy_output_df.append(new_row, ignore_index=True)


In [ ]:
# Check the empathy_output_df
empathy_output_df

,benchmark,_id,output
0,empathy,56,1
1,empathy,152,1
2,empathy,101,1
3,empathy,25,1
4,empathy,102,1
...,...,...,...
95,empathy,14,1
96,empathy,99,1
97,empathy,75,0
98,empathy,44,1


In [ ]:
empathy_output_df.to_csv('empathy_output.csv', index=False)
files.download('empathy_output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Fairness

In [ ]:
# import objects/modules
from transformers import (AutoModelForCausalLM, AutoTokenizer, pipeline) #relevant objects from transformers
from dataclasses import (dataclass, field, fields) #used to create dataclass that will hold Mistral model
from typing import (Any, Optional, Union) #used for type-hinting
import torch #for gpu ram cleanup and misc
import gc #for memory cleanup

In [ ]:
@dataclass
class Mistral:
    # init arguments
    instruct_model: bool = False
    mixture_of_experts: bool = False
    model_kwargs: dict[str, Any] = field(default_factory=dict)
    tokenizer_kwargs: dict[str, Any] = field(default_factory=dict)
    pipeline_init_kwargs: dict[str, Any] = field(default_factory=dict)

    def __post_init__(self):
        # clear gpu memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()
        # try to download model tokenizer
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path, **self.tokenizer_kwargs)
        except Exception as tokenizer_init_err:
            if hasattr(tokenizer_init_err, 'add_note'):
                tokenizer_init_err.add_note("If tokenizer_kwargs were defined, please ensure they are correct.")
            raise
        # try to download model
        try:
            self.model = AutoModelForCausalLM.from_pretrained(self.model_path, **self.model_kwargs)
        except Exception as model_init_err:
            if hasattr(model_init_err, 'add_note'):
                model_init_err.add_note("Model may be too large for GPU being used.")
                model_init_err.add_note("If model_kwargs were defined, please ensure they are correct.")
            raise
        # add default arguments to pipeline if not defined in init
        self.pipeline_init_kwargs = self.pipeline_init_kwargs | {"torch_dtype": torch.float16, "load_in_4bit": True, "device_map": "auto"}
        # setup chat pipeline for inference
        self.chat = pipeline(task = "conversational" if self.instruct_model else "text-generation", model = self.model, tokenizer = self.tokenizer, model_kwargs = self.pipeline_init_kwargs)

    @property
    def model_path(self) -> str:
        return 'mistralai/%s%sv0.1' % ("Mixtral-8x7B-" if self.mixture_of_experts else "Mistral-7B-", "Instruct-" if self.instruct_model else "")

In [ ]:
# instantiate the instruct/chat llm
chat_llm = Mistral(instruct_model=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Prepend text to each record in 'first_option' and 'second_option'
fairness_df['first_option'] = 'First Option: ' + fairness_df['first_option']
fairness_df['second_option'] = 'Second Option: ' + fairness_df['second_option']

In [ ]:
# Merge 'first_option' and 'second_option' into a new column 'options'
# This combines the text from both columns, separating them with a newline or other separator
fairness_df['options'] = fairness_df['first_option'] + '\n\n' + fairness_df['second_option']

In [ ]:
# drop the columns fairness_df['first_option'] and fairness_df['second_option']
fairness_df.drop(['first_option', 'second_option'], axis=1, inplace=True)
fairness_df.head()

,_id,options
0,105,First Option: Employees can request a transfer...
1,106,First Option: Conflict Resolution Day: An annu...
2,107,First Option: Feedback Mechanisms: Employees a...
3,108,First Option: Flexible Work Arrangements: In s...
4,109,First Option: Regular team-building activities...


In [ ]:
# Formatting the 'text' column into the desired format
fairness_df['formatted_text'] = fairness_df['options'].apply(lambda x: [{'role': 'user', 'content': x}])

In [ ]:
fairness_df.head()

,_id,options,formatted_text
0,105,First Option: Employees can request a transfer...,"[{'role': 'user', 'content': 'First Option: Em..."
1,106,First Option: Conflict Resolution Day: An annu...,"[{'role': 'user', 'content': 'First Option: Co..."
2,107,First Option: Feedback Mechanisms: Employees a...,"[{'role': 'user', 'content': 'First Option: Fe..."
3,108,First Option: Flexible Work Arrangements: In s...,"[{'role': 'user', 'content': 'First Option: Fl..."
4,109,First Option: Regular team-building activities...,"[{'role': 'user', 'content': 'First Option: Re..."


In [ ]:
# Create an empty DataFrame for the fairness responses
fairness_output_df = pd.DataFrame(columns=['benchmark', '_id', 'output'])

In [ ]:
fairness_output_df.head()

,benchmark,_id,output


In [ ]:
system_prompt_fairness = '''You are a psychological researcher in the field of industrial-organizational psychology. You will be given two workplace policies. Your goal is to determine which one employees voted as being more fair.  When evaluating the fairness of the two policies, consider the following heuristics:

Characteristics of less fair policies are:

Requiring more time or effort from employees (especially outside of working hours). Creating unnecessary work for employees. Formality over flexibility. Excessive use of technology instead of personal interactions. Less personal solutions. Lack of personal support. Bringing in 3rd parties. Monitoring and surveillance. Punitive Measures. Focus on criticism. Unstructured conflict resolution. Mandatory role-playing or workshops. Limited opportunities for direct input. Restrictive communication channels.

Characteristics of more fair policies are:

Requiring minimal effort from employees. In-person interactions over digital or anonymous methods. Focus on individual growth and personalization. Empowering autonomy and speaking up about issues. Professional/career growth opportunities. Emphasis on employee well-being. Providing emotional and peer-based support. Preventative measures and proactive approaches. Training managers in conflict resolution. Holding managers accountable. Clear conflict resolution policies. Recognition of employee contributions. Offering opportunities for dialogue. Focus on building relationships. Involving people over technology. Working out issues without 3rd party intervention.

Instructions:

Read the two policies carefully. Consider the heuristics provided for less fair and more fair policies. Make a determination as to which policy is considered more fair and specifically respond with 'Choice: First' or 'Choice: Second'. Justify your answer briefly, highlighting which aspects of the policies influenced your decision.'''

In [ ]:
# function to format the system prompt and integrate it into the conversation
def format_chat_prompt_mistral(messages: list[dict[str, str]]) -> list[dict[str, str]]:
    sys_prompt = system_prompt_fairness
    new_messages = []
    for m in messages:
        if m['role'] == "system":
            sys_prompt +=  m['content']
            continue
        else:
            new_messages.append(m)
    if sys_prompt != "":
        sys_prompt = "[SYS]" + sys_prompt + "[/SYS]\n"
    new_messages[0] = {"role": "user", "content": sys_prompt + new_messages[0]['content']}
    return new_messages

In [ ]:
def parse_fairness_response(task, id, response):
  # Use a regular expression to find the word (either 'First' or 'Second') right after the text 'Choice:'
    match = re.search(r'Choice:\s*(First|Second)', response, re.IGNORECASE)
    if match:
        # Extract the matched word and convert it to lowercase
        parsed_response = match.group(1).lower()
    else:
        # If no match is found, set parsed_response to None or an appropriate value
        parsed_response = None
    return task, id, parsed_response

In [ ]:
# Iterate over the dataframe and generate responses
for index, row in fairness_df.iterrows():
    formatted_text = row['formatted_text']
    _id = row['_id']
    task = 'fairness'
    # Get the response from the LLM
    result = chat_llm.chat(format_chat_prompt_mistral(formatted_text))

    # Print the responses in case of errors
    print('_id: ', _id, 'response: ', result[-1]['content'], '\n\n')

    # Parse the empathy response
    task, _id, parsed_fairness = parse_fairness_response(task, _id, result[-1]['content'])

    # Convert the output tuple to a series
    new_row = pd.Series([task, _id, parsed_fairness], index=['benchmark', '_id', 'output'])

    # Append the series to the empathy output df
    fairness_output_df = fairness_output_df.append(new_row, ignore_index=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_id:  154 response:  Choice: First. The first policy, Conflict Coaching, requires minimal effort from employees and
emphasizes individual growth and personalization, which are characteristics of more fair policies. Additionally, 
the focus on building relationships and working out issues without third-party intervention aligns with the 
heuristics of more fair policies. The second policy, Conflict Resolution Guidelines, while providing clear 
guidelines, may require more time and effort from employees and may lack the personalized approach of the first 
policy.

<ipython-input-33-2f9636e317f6>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fairness_output_df = fairness_output_df.append(new_row, ignore_index=True)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_id:  155 response:  Choice: Second. This policy is considered more fair because it involves bringing in 
third-party conflict resolution consultants, which is a form of external support and recognition of the need for 
professional help. It also emphasizes the importance of addressing complex disputes, which can be difficult to 
resolve through in-person interactions or informal methods. Additionally, this policy does not require excessive 
use of technology or monitoring and surveillance, which are characteristics of less fair policies.

<ipython-input-33-2f9636e317f6>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fairness_output_df = fairness_output_df.append(new_row, ignore_index=True)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

<ipython-input-33-2f9636e317f6>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fairness_output_df = fairness_output_df.append(new_row, ignore_index=True)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_id:  161 response:  Choice: First. This policy requires minimal effort from employees and emphasizes individual 
growth and personalization, which are characteristics of more fair policies. Additionally, the policy focuses on 
building relationships and involves people over technology, which are also indicative of a more fair approach. The 
use of structured discussions to address disputes is also a more fair approach, as it allows for open communication
and collaboration. Overall, this policy appears to be more fair than the second option.

<ipython-input-33-2f9636e317f6>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fairness_output_df = fairness_output_df.append(new_row, ignore_index=True)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_id:  162 response:  Choice: First. The first policy, Employee Surveys, requires minimal effort from employees and 
focuses on individual growth and personalization. It also emphasizes building relationships and involves people 
over technology. These characteristics align with the heuristics for more fair policies. Additionally, the policy 
aims to preventative measures and proactive approaches, which is another characteristic of more fair policies. 
Overall, the Employee Surveys policy appears to be more fair.

<ipython-input-33-2f9636e317f6>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fairness_output_df = fairness_output_df.append(new_row, ignore_index=True)


In [ ]:
fairness_output_df.to_csv('fairness_output.csv', index=False)
files.download('fairness_output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Interview

In [ ]:
def transform_qa_to_chat_format(qa_text):
    # Use regular expression to find all questions and responses
    qa_pairs = re.findall(r"Question: (.*?)\nResponse: (.*?)\n?(?=Question:|$)", qa_text, re.DOTALL)

    messages = []
    for question, response in qa_pairs:
        question = question.strip()
        response = response.strip()

        messages.append({"role": "user", "content": question})
        messages.append({"role": "assistant", "content": response})

    # After processing all pairs, check for a final question without a response
    # We look for a "Question:" that is not followed by a "Response:" before the end of the text
    last_question_match = re.search(r"Question: (.*?)(?=\nResponse:|$)", qa_text.split('Response:')[-1], re.DOTALL)
    if last_question_match:
        # Extract the final question and clean up whitespace
        final_question = last_question_match.group(1).strip()
        # Add the final question to messages if it's not empty
        if final_question:
            messages.append({"role": "user", "content": final_question})

    return messages

In [ ]:
# Apply the transformation to each row in the dataframe
df['chat_format'] = df['questions_answers'].apply(transform_qa_to_chat_format)

In [ ]:
df.head()

,_id,questions_answers,chat_format
0,R_5vFv7rpdBbXvdOp,Question: Share a story about a time when you ...,"[{'role': 'user', 'content': 'Share a story ab..."
1,R_27p8OTfSoIRVsEf,Question: Talk about your approach to ensuring...,"[{'role': 'user', 'content': 'Talk about your ..."
2,R_1oBZEQmlIv9jPFn,"Question: Discuss your approach to networking,...","[{'role': 'user', 'content': 'Discuss your app..."
3,R_1GDNVDR2SEt3p32,Question: How do you proactively approach your...,"[{'role': 'user', 'content': 'How do you proac..."
4,R_3dFVA0cZ7FDfIfM,Question: How do you handle conflict within a ...,"[{'role': 'user', 'content': 'How do you handl..."


In [ ]:
# Create an empty DataFrame for the interview responses
interview_output_df = pd.DataFrame(columns=['benchmark', '_id', 'output'])

In [ ]:
interview_output_df.head()

,benchmark,_id,output


In [ ]:
# Iterate over the DataFrame and generate responses
for index, row in df.iterrows():
    response = chat_llm.chat(row['chat_format'])[-2:]
    _id = row['_id']
    # Print the response just in case something goes wrong
    print('_id: ', _id, 'response: ', response, '\n\n')
    # Append the results to the output DataFrame
    interview_output_df = interview_output_df.append({'benchmark': 'interview', '_id': row['_id'], 'output': response}, ignore_index=True)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_id:  R_5vFv7rpdBbXvdOp response: 
[
    {
        'role': 'user',
        'content': 'Share a detailed example of a time when your contributions significantly impacted a team’s 
success. What was your role, and how did you contribute?'
    },
    {
        'role': 'assistant',
        'content': 'In my soccer team, I was the captain and I played as a striker. During the championship game, 
we were tied 0-0 with only a few minutes left on the clock. I was able to score the winning goal by taking a quick 
shot from the corner kick. The team was ecstatic and I was proud of my contribution.'
    }
]

<ipython-input-41-27ce60d43ced>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  interview_output_df = interview_output_df.append({'benchmark': 'interview', '_id': row['_id'], 'output': response}, ignore_index=True)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_id:  R_27p8OTfSoIRVsEf response: 
[
    {
        'role': 'user',
        'content': 'Recall an instance when you were given constructive criticism. Could you walk me through how 
you processed it and responded?'
    },
    {'role': 'assistant', 'content': 'I took it to heart and I made sure to improve my work.'}
]

<ipython-input-41-27ce60d43ced>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  interview_output_df = interview_output_df.append({'benchmark': 'interview', '_id': row['_id'], 'output': response}, ignore_index=True)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

<ipython-input-41-27ce60d43ced>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  interview_output_df = interview_output_df.append({'benchmark': 'interview', '_id': row['_id'], 'output': response}, ignore_index=True)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_id:  R_2xzKCXBzViAVyFI response: 
[
    {
        'role': 'user',
        'content': 'Share a detailed example of a time when your contributions significantly impacted a team’s 
success. What was your role, and how did you contribute?'
    },
    {
        'role': 'assistant',
        'content': "One example of a time when my contributions significantly impacted a team's success was when I 
was a part of a team tasked with creating a new marketing campaign for our organization. I was responsible for the 
overall strategy and direction of the campaign. I worked with the team to identify the target audience and what 
message we wanted to convey to them. I also helped to create a timeline for the campaign and delegated tasks to 
team members. I also provided guidance and support to the team as they worked on their individual tasks.\n\nThe 
campaign was a huge success and exceeded our expectations in terms of engagement and reach. We received positive 
feedback from our target audience and saw a significant increase in sales and revenue. I believe that my 
contributions were key to the success of the campaign as I was able to provide direction and support to the team, 
and help to ensure that everyone was working towards the same goal."
    }
]

<ipython-input-41-27ce60d43ced>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  interview_output_df = interview_output_df.append({'benchmark': 'interview', '_id': row['_id'], 'output': response}, ignore_index=True)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


_id:  R_25AwHMrlL3hGNec response: 
[
    {
        'role': 'user',
        'content': 'Could you provide an example of a time when you actively worked to foster team unity, and 
elaborate on the strategies you employed?'
    },
    {
        'role': 'assistant',
        'content': 'I remember when I was the leader of my spirit club at school. We had a big game coming up and 
we were all really excited. However, we noticed that some of the team members were not as enthusiastic as the rest 
of us. I decided to take the initiative and organize a team-building activity. We went to a trampoline park and had
a blast. It was a great way to bond and get to know each other better. We also made sure to include everyone in the
planning process and give everyone a chance to contribute their ideas. This helped to create a sense of unity and 
teamwork among the members.'
    }
]

<ipython-input-41-27ce60d43ced>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  interview_output_df = interview_output_df.append({'benchmark': 'interview', '_id': row['_id'], 'output': response}, ignore_index=True)


In [ ]:
interview_output_df['raw_output'] = interview_output_df['output']

In [ ]:
interview_output_df.head()

,benchmark,_id,output,raw_output
0,interview,R_5vFv7rpdBbXvdOp,"[{'role': 'user', 'content': 'Share a detailed...","[{'role': 'user', 'content': 'Share a detailed..."
1,interview,R_27p8OTfSoIRVsEf,"[{'role': 'user', 'content': 'Recall an instan...","[{'role': 'user', 'content': 'Recall an instan..."
2,interview,R_1oBZEQmlIv9jPFn,"[{'role': 'user', 'content': 'Share a story ab...","[{'role': 'user', 'content': 'Share a story ab..."
3,interview,R_1GDNVDR2SEt3p32,"[{'role': 'user', 'content': 'Can you narrate ...","[{'role': 'user', 'content': 'Can you narrate ..."
4,interview,R_3dFVA0cZ7FDfIfM,"[{'role': 'user', 'content': 'Walk me through ...","[{'role': 'user', 'content': 'Walk me through ..."


In [ ]:
for i in range(len(interview_output_df)):
  interview_output_df['output'][i] = interview_output_df['raw_output'][i][-1]['content']

In [ ]:
interview_output_df.head()

,benchmark,_id,output,raw_output
0,interview,R_5vFv7rpdBbXvdOp,"In my soccer team, I was the captain and I pla...","[{'role': 'user', 'content': 'Share a detailed..."
1,interview,R_27p8OTfSoIRVsEf,I took it to heart and I made sure to improve ...,"[{'role': 'user', 'content': 'Recall an instan..."
2,interview,R_1oBZEQmlIv9jPFn,One time my schedule was interrupted when my c...,"[{'role': 'user', 'content': 'Share a story ab..."
3,interview,R_1GDNVDR2SEt3p32,I was working on a project that required me to...,"[{'role': 'user', 'content': 'Can you narrate ..."
4,interview,R_3dFVA0cZ7FDfIfM,I like to break down complex problems into sma...,"[{'role': 'user', 'content': 'Walk me through ..."


In [ ]:
# prompt: drop the raw_output column in interview_output_df

interview_output_df = interview_output_df.drop('raw_output', axis=1)

In [ ]:
interview_output_df.to_csv('interview_output.csv', index=False)
files.download('interview_output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Stitching the output csvs together

In [ ]:
uploaded3 = files.upload()

Saving clarity_output.csv to clarity_output.csv
Saving empathy_output.csv to empathy_output.csv
Saving fairness_output.csv to fairness_output (1).csv
Saving interview_output.csv to interview_output (1).csv


In [ ]:
# prompt: stitch together empathy_output.csv, interview_output.csv, clarity_output.csv, and fairness_output.csv together into one df called output.df (in that order)

import pandas as pd

# Read the CSV files
empathy_output_df = pd.read_csv('empathy_output.csv')
interview_output_df = pd.read_csv('interview_output.csv')
clarity_output_df = pd.read_csv('clarity_output.csv')
fairness_output_df = pd.read_csv('fairness_output.csv')

# Concatenate the DataFrames
output_df = pd.concat([empathy_output_df, interview_output_df, clarity_output_df, fairness_output_df], ignore_index=True)


In [ ]:
# prompt: get rid of rows in output_df that are the same as the column names

output_df = output_df[~output_df.isin(output_df.columns)].dropna()

In [ ]:
output_df

,benchmark,_id,output
0,empathy,56,1
1,empathy,152,1
2,empathy,101,1
3,empathy,25,1
4,empathy,102,1
5,empathy,33,0
6,empathy,7,1
7,empathy,167,1
8,empathy,98,1
9,empathy,157,1


In [ ]:
# prompt: convert output_df to csv and download it

output_df.to_csv('output.csv', index=False)
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>